## Work
1. 請嘗試將 preproc_x 替換成以每筆資料的 min/max 進行標準化至 -1 ~ 1 間，再進行訓練
2. 請嘗試將 mlp 疊更深 (e.g 5~10 層)，進行訓練後觀察 learning curve 的走勢
3. (optional) 請改用 GPU 進行訓練 (如果你有 GPU 的話)，比較使用 CPU 與 GPU 的訓練速度

In [1]:
## 
"""
Your code here (optional)
確認硬體資源
"""


In [1]:
import os
import keras

# 請嘗試設定 GPU：os.environ


Using TensorFlow backend.


In [2]:
train, test = keras.datasets.cifar10.load_data()

In [3]:
## 資料前處理
"""
Your code here
"""

import numpy as np

def preproc_x(x, flatten=True):
    x = x / abs(np.max(x))
    if flatten:
        x = x.reshape((len(x), -1))
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y    

In [4]:
x_train, y_train = train
x_test, y_test = test

# Preproc the inputs
x_train = preproc_x(x_train, True)
x_test = preproc_x(x_test)

# Preprc the outputs
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [5]:
"""Code Here
建立你的神經網路
"""
def build_mlp(input_shape, output_units=10, num_neurons=[512, 256, 128, 64, 32]):
    input_layer = keras.layers.Input(input_shape)
    
    for i, n_units in enumerate(num_neurons):
        if i == 0:
            x = keras.layers.Dense(units=n_units, activation="relu", name="hidden_layer"+str(i+1))(input_layer)
        else:
            x = keras.layers.Dense(units=n_units, activation="relu", name="hidden_layer"+str(i+1))(x)
    
    out = keras.layers.Dense(units=output_units, activation="softmax", name="output")(x)
    
    model = keras.models.Model(inputs=[input_layer], outputs=[out])
    return model
 
model = build_mlp(input_shape=x_train.shape[1:])
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3072)              0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 512)               1573376   
_________________________________________________________________
hidden_layer2 (Dense)        (None, 256)               131328    
_________________________________________________________________
hidden_layer3 (Dense)        (None, 128)               32896     
_________________________________________________________________
hidden_layer4 (Dense)        (None, 64)                8256      
_________________________________________________________________
hidden_layer5 (Dense)        (None, 32)                2080      
_________________________________________________________________
output (Dense)               (None, 10)                330 

In [6]:
## 超參數設定
LEARNING_RATE = 0.001
EPOCHS = 100
BATCH_SIZE = 256

In [7]:
optimizer = keras.optimizers.Adam(lr=LEARNING_RATE)
model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer=optimizer)

In [8]:
model.fit(x_train, y_train, 
          epochs=EPOCHS, 
          batch_size=BATCH_SIZE, 
          validation_data=(x_test, y_test), 
          shuffle=True)

Train on 50000 samples, validate on 10000 samples
Epoch 1/100
50000/50000 [==============================] - 9s 177us/step - loss: 1.9987 - accuracy: 0.2571 - val_loss: 1.8231 - val_accuracy: 0.3166
Epoch 2/100
50000/50000 [==============================] - 9s 172us/step - loss: 1.7586 - accuracy: 0.3632 - val_loss: 1.7310 - val_accuracy: 0.3756
Epoch 3/100
50000/50000 [==============================] - 9s 175us/step - loss: 1.6566 - accuracy: 0.4051 - val_loss: 1.6244 - val_accuracy: 0.4235
Epoch 4/100
50000/50000 [==============================] - 8s 166us/step - loss: 1.5983 - accuracy: 0.4274 - val_loss: 1.5462 - val_accuracy: 0.4495
Epoch 5/100
50000/50000 [==============================] - 8s 164us/step - loss: 1.5497 - accuracy: 0.4442 - val_loss: 1.5479 - val_accuracy: 0.4471
Epoch 6/100
50000/50000 [==============================] - 8s 161us/step - loss: 1.5104 - accuracy: 0.4602 - val_loss: 1.5298 - val_accuracy: 0.4533
Epoch 7/100
50000/50000 [==============================]

50000/50000 [==============================] - 9s 183us/step - loss: 0.6413 - accuracy: 0.7712 - val_loss: 1.8772 - val_accuracy: 0.5083
Epoch 56/100
50000/50000 [==============================] - 9s 182us/step - loss: 0.6354 - accuracy: 0.7723 - val_loss: 1.8578 - val_accuracy: 0.5154
Epoch 57/100
50000/50000 [==============================] - 9s 180us/step - loss: 0.6018 - accuracy: 0.7843 - val_loss: 1.9609 - val_accuracy: 0.5079
Epoch 58/100
50000/50000 [==============================] - 9s 180us/step - loss: 0.5849 - accuracy: 0.7894 - val_loss: 1.9929 - val_accuracy: 0.5116
Epoch 59/100
50000/50000 [==============================] - 9s 181us/step - loss: 0.5933 - accuracy: 0.7884 - val_loss: 1.9881 - val_accuracy: 0.5025
Epoch 60/100
50000/50000 [==============================] - 9s 188us/step - loss: 0.5735 - accuracy: 0.7931 - val_loss: 2.0343 - val_accuracy: 0.5064
Epoch 61/100
50000/50000 [==============================] - 9s 187us/step - loss: 0.5597 - accuracy: 0.8010 - val

In [9]:
import matplotlib.pyplot as plt


train_loss = model.history.history["loss"]
valid_loss = model.history.history["val_loss"]

train_acc = model.history.history["accuracy"]
valid_acc = model.history.history["val_accuracy"]

plt.plot(range(len(train_loss)), train_loss, label="train loss")
plt.plot(range(len(valid_loss)), valid_loss, label="valid loss")
plt.legend()
plt.title("Loss")
plt.show()

plt.plot(range(len(train_acc)), train_acc, label="train accuracy")
plt.plot(range(len(valid_acc)), valid_acc, label="valid accuracy")
plt.legend()
plt.title("Accuracy")
plt.show()

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>